In [2]:
import numpy as np # arrays & loading data

from sklearn.model_selection import train_test_split # we will split the data in 3 (training + counter variation + testing data)
from sklearn.preprocessing import StandardScaler # z-score normalization & polynomianls classes 

import tensorflow as tf # for building and training neural networks